This script is to filter the data downloaded by getData.ipynb
required data in json file
will store filtered data into .xlsx

In [10]:
import json
from typing import cast
import numpy as np
import time
from collections import Counter
from os import times
import matplotlib.pyplot as plt
import openpyxl
from numpy.lib.shape_base import column_stack
from scipy.interpolate import interp1d

# define data-fetch file
wb = openpyxl.Workbook()
carno = "HMZABAAH9MF014494_412_512"
ws = wb.create_sheet(carno)
# data json file         
filename = "../../data/all-info/"+carno+".json" 
# load json 
with open(filename) as file_obj:
    res = json.load(file_obj)       

In [11]:
soc = [0,2,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,98,100]
ocv = [2737.5,3085.5,3180,3209.3,3223.1,3250.4,3266.7,3282.1,3289.2,3289.9,3290.6,3291.3,3293.2,3304.1,3329.6,3330.3,3331,3332,3332.9,3334.1,3334.5,3365,3443.2]

get_ocv = interp1d(soc,ocv,kind='linear')

the code below only filter the data into time,current and voltage1-198



In [19]:
# start from second row
num = 2

for keys in res:
    # save cell voltage
    Vol = []
    for i in range(1, 10):
        try:
            Vol.append(
                keys["_source"]["canData"]["data"]["CellVoltage_0" + str(i)]
            )
        except:
            print(str(i) + "th dose not exist")
            continue
    for i in range(10, 199):
        try:
            Vol.append(
                keys["_source"]["canData"]["data"]["CellVoltage_" + str(i)]
            )
        except:
            print(str(i) + "th dose not exist")
    if sum(Vol) != 0 and Vol.count(8191) == 0:     #8191 is error code.
        vnum = 1
        for v in Vol:
            ws.cell(row=num, column=vnum + 3).value = v
            # iterate all cellvoltage_i
            vnum += 1

        try:
            # save time, current and soc
            ws.cell(row=num, column=1).value = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(int(keys["_source"]["travelTime"])/1000))
            ws.cell(row=num, column=2).value = keys["_source"]["canData"]["data"]["VCU_BMSPackCurrent"]
            ws.cell(row=num, column=3).value = keys["_source"]["canData"]["data"]["BMSSOC"]   
            ws.cell(row=num, column=4).value = keys["_source"]["canData"]["data"]["VCU_BMSChargeState"]
            # ws.cell(row=num, column=5).value = get_ocv(keys["_source"]["canData"]["data"]["BMSSOC"])   
            ocv = get_ocv(keys["_source"]["canData"]["data"]["BMSSOC"])
            ws.cell(row=num, column=5).value = ocv
        except:
            continue
        num += 1  # increment of row
    else:
        print("8191 error!")

# title "CellVoltage1-198"
for i in range(1, 199):
    ws.cell(row=1, column=i + 5).value = "CellVoltage_" + str(i) 
# time, current and soc to file 
ws.cell(row=1, column=1).value = "time"         
ws.cell(row=1, column=2).value = "current"     
ws.cell(row=1, column=3).value = "BMSSOC"
ws.cell(row=1, column=4).value = "VCU_BMSChargeState"
ws.cell(row=1, column=5).value = "ocv"



In [20]:
#save in xlsx
wb.save("../../data/filt-data/"+carno+".xlsx")
print("success") 

success
